In [1]:
import time
import os
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ

In [24]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/Kate's/"
csv_filename = 'raw_velocity_data_JJ.csv'

# load the all-data CSV dir
df = pd.read_csv(data_dir + csv_filename)
df = pd.DataFrame(df)

enable_photoZ_interaction = True
initialize_photoZ = False

skip_recs = [] # ['02-01-05_L23_interlaminar_intralaminar', '03-01-06_L4_intralaminar']

In [21]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir)
    if initialize_photoZ:
        aPhz.prepare_photoZ()

In [32]:
if not enable_photoZ_interaction:
    skip_recs = []
for subdir, dirs, files in os.walk(data_dir):
    if subdir in skip_recs:
        continue
    else:
        skip_recs.append(subdir)
    zda_file = None
    roi_files = []
    roi_files_spare = []
    
    # locate important file names
    for f in files:
        if f.endswith('.zda'):
            zda_file = f
        if f.startswith('rois_'):
            if f.endswith('laminar.dat'):
                roi_files.append(f)
            else:
                roi_files_spare.append(f)
        if zda_file is not None and len(roi_files) == 2:
            break
    if len(roi_files) < 1:
        roi_files = roi_files_spare
        if len(roi_files) > 0:
            roi_files = [roi_files[0]]
        
    if zda_file is not None:
        rec_id = zda_file.split('.')[0]
        # alias the rec_id
        if '_aka_' in subdir:
            rec_id = subdir.split('_aka_')[0][-8:].replace("-", "_")
            print("aliased:", rec_id)
            
        aPhz = AutoPhotoZ(data_dir=subdir)
        
        # open the PhotoZ file
        if enable_photoZ_interaction:
            aPhz.select_PhotoZ()
            
            print("\n\nOpening", zda_file)
            aPhz.open_zda_file(subdir + "/" + zda_file)
        
        date = subdir.split("/")[-1].split('\\')[0].replace('-', '_')
        date = date[5:] + '_' + date[:4]
        print(subdir)
        for roi_file in roi_files:
            print(roi_file)
            
            # open roi file
            if enable_photoZ_interaction:
                aPhz.select_roi_tab()
                aPhz.open_roi_file(roi_file)
            
            # Now load the data from the CSV needed to know the monosynaptic window
            lam_type = None
            lam_type_subset = df[(df['Date'] == date) 
                               & (df['Slice_Loc_Run'] == rec_id)]
            lam_type = list(lam_type_subset.Laminar.unique())

            
            if 'interlaminar' in roi_file and any(["Inter" in x for x in lam_type]):
                lam_type = ['Inter', 'Inter_L23', 'Inter_L5']
            elif 'intralaminar' in roi_file and "Intra" in lam_type:
                lam_type = ['Intra']
            elif 'interlaminar' in subdir and any(["Inter" in x for x in lam_type]):
                lam_type = ['Inter', 'Inter_L23', 'Inter_L5']
            elif 'intralaminar' in subdir and "Intra" in lam_type:
                lam_type = ['Intra']
            else:
                print("For " + " ".join(lam_type) + " for roi_file " + roi_file + " \n\tsubdir " + subdir + " rec " + rec_id)
                raise Exception("What is the laminar type? Unknown")
            mono_window_full = df[(df['Date'] == date) 
                             & (df['Laminar'].isin(lam_type))
                             & (df['Slice_Loc_Run'] == rec_id)]
            mono_window = pd.DataFrame(mono_window_full,
                                       columns = ['Date', 'Laminar',
                                                'Mono_Resp_Start_Time',
                                                'Mono_Resp_End_Time'])
            if mono_window.size < 1:
                raise Exception("(No row) Check laminar value " + " ".join(lam_type) + " for date " + date + " rec " + rec_id)
                continue
            mono_window = mono_window.drop_duplicates()
            if mono_window.shape[0] != 1:
                print(mono_window, mono_window.shape)
                raise Exception("Check laminar value " + " ".join(lam_type) + " for date " + date + " rec " + rec_id)
            
            # calculate and set measure window
            start = int(mono_window['Mono_Resp_Start_Time'] * 2)
            width = int(mono_window['Mono_Resp_End_Time'] * 2 - start)
            if enable_photoZ_interaction:
                aPhz.set_measure_window(start, width)
            else:
                print("Measure window:", start, width)
                
            # Save values to file via PhotoZ: monosynaptic latency, max amp, SNR, peak time
            latency_filename = subdir + "/" + "mono_latency_" + lam_type[0] + '.dat'
            print("Latency file dst:", latency_filename)
            if enable_photoZ_interaction:
                aPhz.select_latency_trace_value()
                aPhz.save_trace_values(latency_filename)
                
            maxamp_filename = subdir + "/" + "mono_maxamp_" + lam_type[0] + '.dat'
            print("maxamp_filename dst:", maxamp_filename)
            if enable_photoZ_interaction:
                aPhz.select_maxamp_trace_value()
                aPhz.save_trace_values(maxamp_filename)
                
            snr_filename = subdir + "/" + "mono_snr_" + lam_type[0] + '.dat'
            print("snr_filename dst:", snr_filename)
            if enable_photoZ_interaction:
                aPhz.select_SNR_trace_value()
                aPhz.save_trace_values(snr_filename)
                
            peaktime_filename = subdir + "/" + "mono_peaktime_" + lam_type[0] + '.dat'
            print("peaktime_filename dst:", peaktime_filename)
            if enable_photoZ_interaction:
                aPhz.select_peaktime_trace_value()
                aPhz.save_trace_values(peaktime_filename)
            
            intermed_files = [latency_filename, maxamp_filename, snr_filename, peaktime_filename]
            col_names = ['Latency_Mono_Raw', 'Amp_Mono', 'SNR_Mono', 'Peak_Time_Mono']
            
            print("BEFORE WRITING")
            print(df[(df['Date'] == date) 
                   & (df['Laminar'].isin(lam_type))
                   & (df['Slice_Loc_Run'] == rec_id)][col_names])
            
            # open these various files and store them to the pandas dataframe
            for i in range(len(intermed_files)):
                intermed_file = intermed_files[i]
                col_name = col_names[i]
                intermed_df = pd.read_csv(intermed_file,
                                         sep='\t',
                                         header=None,
                                         names=['Index',  'Values'])

                row_indices = df.index[(df['Date'] == date) 
                 & (df['Laminar'].isin(lam_type))
                 & (df['Slice_Loc_Run'] == rec_id)].tolist()

                if intermed_df['Values'].shape[0] == len(row_indices):
                    for j in range(len(row_indices)):
                        row_idx = row_indices[j]
                        df.loc[row_idx,[col_name]] = intermed_df['Values'][j]
                    print(intermed_df['Values'])
                    print(df.loc[row_indices,[col_name]])
                else:
                    print("Could not write values for " + " ".join(lam_type) 
                          + " for date " + date + " rec " + rec_id
                          + "col name" + col_name)
                    print(df[(df['Date'] == date) 
                     & (df['Laminar'].isin(lam_type))
                     & (df['Slice_Loc_Run'] == rec_id)])
                    print(intermed_df['Values'])
            
            # print("\n\nNum empty cells ", df[['Latency_Mono_Raw', 'Amp_Mono', 'SNR_Mono', 'Peak_Time_Mono']].count())
            print("AFTER WRITE")
            print(df[(df['Date'] == date) 
                   & (df['Laminar'].isin(lam_type))
                   & (df['Slice_Loc_Run'] == rec_id)][col_names])
            

C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-02\02-01-05_L23_interlaminar_intralaminar
rois_01_to_20_interlaminar.dat
Measure window: 93 8
Latency file dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-02\02-01-05_L23_interlaminar_intralaminar/mono_latency_Inter.dat
maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-02\02-01-05_L23_interlaminar_intralaminar/mono_maxamp_Inter.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-02\02-01-05_L23_interlaminar_intralaminar/mono_snr_Inter.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-02\02-01-05_L23_interlaminar_intralaminar/mono_peaktime_Inter.dat
BEFORE WRITING
    Latency_Mono_Raw  Amp_Mono  SNR_Mono  Peak_Time_Mono
0            47.2902  5.363650  10.52020            48.5
1            47.2542  5.724150  15.64680            48.5
2            47.3070  5.744380  22.81410            48.5
3            47.5374  5.087080  14.96920            49.0
4            47.4508  4.265660  14.60350     

    Latency_Mono_Raw  Amp_Mono  SNR_Mono  Peak_Time_Mono
14           47.2686  6.033900  15.93790            48.5
15           47.1903  6.384060  20.27610            48.5
16           47.4426  6.259390  14.63830            49.0
17           47.5169  6.030880  15.05070            49.0
18           47.8148  5.282070  15.56070            49.5
19           48.0929  4.653680  16.45060            49.5
20           48.1602  3.869940  10.63430            49.5
21           48.0159  3.290400  13.21140            49.5
22           47.7570  2.641030   8.76726            52.0
23           48.4042  2.568710   7.37071            50.0
24           48.0436  1.982850   5.37712            50.0
25           48.7561  1.796110   5.85724            50.5
26           49.8284  1.955960   6.27155            52.0
27           50.4213  1.642240   5.17633            52.0
28           50.2588  0.673868   2.14855            52.0
29           50.1679  1.117460   2.40110            52.0
30           49.5586  1.307210 

77  0.905608
0     49.5
1     49.5
2     99.0
3     99.0
4     49.5
5     49.5
6     99.0
7     99.0
8     49.5
9     49.5
10    99.0
11    99.0
12    49.5
13    99.0
14    49.5
15    99.0
16    49.5
17    99.0
18    49.5
19    49.5
20    49.5
21    49.5
22    49.5
23    99.0
24    49.5
Name: Values, dtype: float64
    Peak_Time_Mono
53            49.5
54            49.5
55            99.0
56            99.0
57            49.5
58            49.5
59            99.0
60            99.0
61            49.5
62            49.5
63            99.0
64            99.0
65            49.5
66            99.0
67            49.5
68            99.0
69            49.5
70            99.0
71            49.5
72            49.5
73            49.5
74            49.5
75            49.5
76            99.0
77            49.5
AFTER WRITE
    Latency_Mono_Raw  Amp_Mono  SNR_Mono  Peak_Time_Mono
53           49.4313  0.086705  0.465328            49.5
54           49.2260  0.347578  1.031230            49.5
55    

0     49.5
1     49.5
2     49.5
3     49.5
4     99.0
5     99.0
6     99.0
7     99.0
8     99.0
9     99.0
10    99.0
Name: Values, dtype: float64
     Peak_Time_Mono
94             49.5
95             49.5
96             49.5
97             49.5
98             99.0
99             99.0
100            99.0
101            99.0
102            99.0
103            99.0
104            99.0
AFTER WRITE
     Latency_Mono_Raw  Amp_Mono  SNR_Mono  Peak_Time_Mono
94            49.0389  0.406374  1.423690            49.5
95            49.2709  0.229637  0.930896            49.5
96            49.4316  0.090808  0.322205            49.5
97            50.1114  0.189661  0.689123            49.5
98            99.0000  0.000000  0.000000            99.0
99            99.0000  0.000000  0.000000            99.0
100           99.0000  0.000000  0.000000            99.0
101           99.0000  0.000000  0.000000            99.0
102           99.0000  0.000000  0.000000            99.0
103           99.0

Name: Values, dtype: float64
     SNR_Mono
117  0.602679
118  0.236245
119  0.386065
120  0.342242
121  0.000000
122  0.000000
123  0.000000
124  0.000000
125  0.952321
126  0.000000
127  0.274112
128  0.000000
129  0.000000
130  0.000000
131  0.000000
132  0.000000
133  1.488580
134  1.132940
0     49.5
1     49.5
2     49.5
3     49.5
4     99.0
5     99.0
6     99.0
7     99.0
8     49.5
9     99.0
10    49.5
11    99.0
12    99.0
13    99.0
14    99.0
15    99.0
16    49.5
17    49.5
Name: Values, dtype: float64
     Peak_Time_Mono
117            49.5
118            49.5
119            49.5
120            49.5
121            99.0
122            99.0
123            99.0
124            99.0
125            49.5
126            99.0
127            49.5
128            99.0
129            99.0
130            99.0
131            99.0
132            99.0
133            49.5
134            49.5
AFTER WRITE
     Latency_Mono_Raw  Amp_Mono  SNR_Mono  Peak_Time_Mono
117           49.2543  0.359

...

0     99.0000
1     49.0816
2     49.0734
3     99.0000
4     99.0000
5     49.4338
6     99.0000
7     99.0000
8     48.6687
9     47.9622
10    99.0000
11    99.0000
12    99.0000
13    99.0000
14    50.5556
15    49.1035
16    99.0000
17    99.0000
18    49.1902
19    99.0000
20    99.0000
21    99.0000
Name: Values, dtype: float64
      Latency_Mono_Raw
1632           99.0000
1633           49.0816
1634           49.0734
1635           99.0000
1636           99.0000
1637           49.4338
1638           99.0000
1639           99.0000
1640           48.6687
1641           47.9622
1642           99.0000
1643           99.0000
1644           99.0000
1645           99.0000
1646           50.5556
1647           49.1035
1648           99.0000
1649           99.0000
1650           49.1902
1651           99.0000
1652           99.0000
1653           99.0000
0     0.000000
1     0.219665
2     0.355201
3     0.000000
4     0.000000
5     0.032932
6     0.000000
7     0.000000
8     0.203588

Measure window: 99 7
Latency file dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-09-06\04-01-04_L23_interlaminar_intralaminar/mono_latency_Intra.dat
maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-09-06\04-01-04_L23_interlaminar_intralaminar/mono_maxamp_Intra.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-09-06\04-01-04_L23_interlaminar_intralaminar/mono_snr_Intra.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-09-06\04-01-04_L23_interlaminar_intralaminar/mono_peaktime_Intra.dat
BEFORE WRITING
      Latency_Mono_Raw  Amp_Mono  SNR_Mono  Peak_Time_Mono
1694               NaN       NaN       NaN             NaN
1695               NaN       NaN       NaN             NaN
1696               NaN       NaN       NaN             NaN
1697               NaN       NaN       NaN             NaN
1698               NaN       NaN       NaN             NaN
1699               NaN       NaN       NaN             NaN
1700               NaN       NaN   

      Peak_Time_Mono
1669            49.5
1670            49.5
1671            49.5
1672            99.0
1673            49.5
1674            49.5
1675            49.5
1676            49.5
1677            99.0
1678            49.5
1679            49.5
1680            49.5
1681            99.0
1682            99.0
1683            99.0
1684            99.0
1685            99.0
1686            99.0
1687            99.0
1688            99.0
1689            99.0
1690            99.0
1691            99.0
1692            99.0
1693            49.5
AFTER WRITE
      Latency_Mono_Raw  Amp_Mono  SNR_Mono  Peak_Time_Mono
1669           48.2831  0.480883  1.397730            49.5
1670           49.3854  0.156709  0.397607            49.5
1671           44.4184  0.410565  1.551990            49.5
1672           99.0000  0.000000  0.000000            99.0
1673           49.4534  0.022808  0.096220            49.5
1674           48.2557  0.451487  1.660290            49.5
1675           51.2391  0.311

      Latency_Mono_Raw  Amp_Mono  SNR_Mono  Peak_Time_Mono
1710           48.7274  0.409708  1.340220            49.0
1711           48.7366  0.233151  0.933202            49.0
1712           98.0000  0.000000  0.000000            98.0
1713           48.9854  0.011448  0.047514            49.0
1714           48.8263  0.059478  0.277067            49.0
1715           48.4558  0.100949  0.699077            49.0
1716           98.0000  0.000000  0.000000            98.0
1717           98.0000  0.000000  0.000000            98.0
1718           98.0000  0.000000  0.000000            98.0
1719           98.0000  0.000000  0.000000            98.0
1720           98.0000  0.000000  0.000000            98.0
1721           98.0000  0.000000  0.000000            98.0
1722           98.0000  0.000000  0.000000            98.0
1723           47.0259  0.146229  1.186500            49.0
1724           48.5237  0.083714  0.833032            49.0
1725           48.6402  0.044369  0.461604            49

      SNR_Mono
1749  0.000000
1750  0.762304
1751  0.044509
1752  0.000000
1753  0.000000
1754  0.000000
1755  0.000000
1756  0.000000
1757  0.000000
1758  0.000000
1759  0.000000
1760  0.000000
1761  0.000000
1762  0.000000
1763  0.000000
1764  0.000000
1765  0.000000
1766  0.000000
1767  0.000000
1768  0.691732
1769  0.257398
1770  0.000000
1771  0.000000
1772  0.000000
0     99.0
1     49.5
2     49.5
3     99.0
4     99.0
5     99.0
6     99.0
7     99.0
8     99.0
9     99.0
10    99.0
11    99.0
12    99.0
13    99.0
14    99.0
15    99.0
16    99.0
17    99.0
18    99.0
19    49.5
20    49.5
21    99.0
22    99.0
23    99.0
Name: Values, dtype: float64
      Peak_Time_Mono
1749            99.0
1750            49.5
1751            49.5
1752            99.0
1753            99.0
1754            99.0
1755            99.0
1756            99.0
1757            99.0
1758            99.0
1759            99.0
1760            99.0
1761            99.0
1762            99.0
1763            9

In [33]:
# write the pandas dataframe back to csv
csv_filename = 'raw_velocity_data_JJ_edited.csv'
df.to_csv(data_dir + csv_filename, index=False)

In [34]:
# Compare KS and JJ csv files
ks_csv_filename = 'raw_velocity_data_KS.csv'
ks_df = pd.read_csv(data_dir + csv_filename)
ks_df = pd.DataFrame(ks_df)

